# Battle of the Neighbourhoods

## Problem

Someone wants to open up a new Italian restaurant. The prospective restaurant owner has picked two possible locations to do so, but he is unsure whether he will able to face the competition in these locations. He needs to know how many restaurants he will be competing with and how many of those are Italian, so he is interested in knowing how many restaurants there are within walking distance (1km). He is also interested in the ratings, since he is going to provide topquality food and is not worried about restaurants with lower scores.

### Questions

- How many restaurants are within a 1 km radius of the prospective location?
- How many of those restaurants are Italian?
- What are the ratings of these restaurants?

### Data

The relevant data can be retrieved from the Foursquare API, as this dataset contains all relevant location information about restaurants, as well their popularity among the public. 

## Exploring location 1

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

In [3]:
CLIENT_ID = 'MO3O4XAX43BL54CUKD1UDB0L53US2E1JZSHT5H52R0I3TQNB' # your Foursquare ID
CLIENT_SECRET = 'N3VLVMZ4OZPD0YYO5C05WNJQORG2OHX0UMBXR1E2550RELOO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = ()
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MO3O4XAX43BL54CUKD1UDB0L53US2E1JZSHT5H52R0I3TQNB
CLIENT_SECRET:N3VLVMZ4OZPD0YYO5C05WNJQORG2OHX0UMBXR1E2550RELOO


In [4]:
# Establishing longitude and latitude of location 1

address = 'Zwanenveld 66, Nijmegen'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print(latitude1, longitude1)

51.8210402 5.8015077


Let's find the restaurants around the location!

In [6]:
search_query = 'Restaurant'
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=MO3O4XAX43BL54CUKD1UDB0L53US2E1JZSHT5H52R0I3TQNB&client_secret=N3VLVMZ4OZPD0YYO5C05WNJQORG2OHX0UMBXR1E2550RELOO&ll=51.8210402,5.8015077&v=20180604&query=Restaurant&radius=1000'

In [7]:
results = requests.get(url).json()
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dflocation1 = json_normalize(venues)
dflocation1.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,502673d6e4b041e2a674a716,Restaurant Santa Maria,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",v-1587885434,False,Rijksweg 26,51.814349,5.802620,"[{'label': 'display', 'lat': 51.8143494490257,...",748,6581,NL,Malden,Gelderland,Nederland,"[Rijksweg 26, 6581 Malden, Nederland]"
1,589ccf1551d19e538d086c7f,Chinees-Indisch Restaurant China Garden,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1587885434,False,Meijhorst 7027,51.811660,5.801110,"[{'label': 'display', 'lat': 51.81166, 'lng': ...",1044,6537 EP,NL,Nijmegen,Gelderland,Nederland,"[Meijhorst 7027, 6537 EP Nijmegen, Nederland]"
2,4c0df9f598102d7f72f6e206,Ziggo,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1587885434,False,Takenhofplein 1,51.825847,5.792291,"[{'label': 'display', 'lat': 51.82584691466280...",829,6538 SZ,NL,Nijmegen,Gelderland,Nederland,"[Takenhofplein 1, 6538 SZ Nijmegen, Nederland]"


In [8]:
dflocation1.shape

(3, 16)

# Clearly, there are not a whole lot of restaurants in the vicinity of this location!

## Let's check the ratings

In [15]:
urls = []
for id in dflocation1['id']:
    ratingsurl = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(id, CLIENT_ID, CLIENT_SECRET, VERSION)
    #for restaurant in ratingsurl:
    urls.append(ratingsurl)
print(urls)

['https://api.foursquare.com/v2/venues/502673d6e4b041e2a674a716?client_id=MO3O4XAX43BL54CUKD1UDB0L53US2E1JZSHT5H52R0I3TQNB&client_secret=N3VLVMZ4OZPD0YYO5C05WNJQORG2OHX0UMBXR1E2550RELOO&v=20180604', 'https://api.foursquare.com/v2/venues/589ccf1551d19e538d086c7f?client_id=MO3O4XAX43BL54CUKD1UDB0L53US2E1JZSHT5H52R0I3TQNB&client_secret=N3VLVMZ4OZPD0YYO5C05WNJQORG2OHX0UMBXR1E2550RELOO&v=20180604', 'https://api.foursquare.com/v2/venues/4c0df9f598102d7f72f6e206?client_id=MO3O4XAX43BL54CUKD1UDB0L53US2E1JZSHT5H52R0I3TQNB&client_secret=N3VLVMZ4OZPD0YYO5C05WNJQORG2OHX0UMBXR1E2550RELOO&v=20180604']


In [26]:
venues = []
for venue in urls:
    result = requests.get(ratingsurl).json()
    venues.append(result)
print(venues)

[{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5ea5386fb4b684001b41783f'}, 'response': {}}, {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5ea53930edbcad001bc1e1e6'}, 'response': {}}, {'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '5ea539b3b57e88001be68709'}, 'response': {}}]


## Unfortunately I exceeded my Foursquare API quota, so I could not check the ratings. Judging from the error, however, my code seems to work.

# Exploring location 2

In [28]:
# Establishing longitude and latitude of location 2

address = 'Tooropstraat 16, Nijmegen'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude2 = location.latitude
longitude2 = location.longitude
print(latitude2, longitude2)

51.8367581 5.8785961


In [30]:
search_query = 'Restaurant'
radius = 1000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=MO3O4XAX43BL54CUKD1UDB0L53US2E1JZSHT5H52R0I3TQNB&client_secret=N3VLVMZ4OZPD0YYO5C05WNJQORG2OHX0UMBXR1E2550RELOO&ll=51.8367581,5.8785961&v=20180604&query=Restaurant&radius=1000'

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea539e777af03001ba4a0d9'},
 'response': {'venues': [{'id': '4b803ffff964a520cd6030e3',
    'name': 'Restaurant Zusje Oost',
    'location': {'address': '20160219',
     'lat': 51.834995228388394,
     'lng': 5.881848335266113,
     'labeledLatLngs': [{'label': 'display',
       'lat': 51.834995228388394,
       'lng': 5.881848335266113}],
     'distance': 297,
     'cc': 'NL',
     'country': 'Nederland',
     'formattedAddress': ['20160219', 'Nederland']},
    'categories': [{'id': '4bf58dd8d48988d1c4941735',
      'name': 'Restaurant',
      'pluralName': 'Restaurants',
      'shortName': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1587886515',
    'hasPerk': False},
   {'id': '5984b70bfe37406ea7b5e21f',
    'name': 'Restaurant Klinkklaar',
    'location': {'address': 'Tooropstraat 59',
     'lat': 51.8368899,
     'lng': 5.8

In [32]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.postalCode,location.city,location.state,location.crossStreet
0,4b803ffff964a520cd6030e3,Restaurant Zusje Oost,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1587886515,False,20160219,51.834995,5.881848,"[{'label': 'display', 'lat': 51.83499522838839...",297,NL,Nederland,"[20160219, Nederland]",NaN,NaN,NaN,NaN
1,5984b70bfe37406ea7b5e21f,Restaurant Klinkklaar,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1587886515,False,Tooropstraat 59,51.836890,5.879140,"[{'label': 'display', 'lat': 51.8368899, 'lng'...",40,NL,Nederland,"[Tooropstraat 59, 6521 NJ Nijmegen, Nederland]",6521 NJ,Nijmegen,Gelderland,NaN
2,4ce55494dfaef04d9d7f6d83,Chinees Restaurant De Mandarijn,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1587886515,False,Tooropstraat 47,51.837077,5.878130,"[{'label': 'display', 'lat': 51.837077, 'lng':...",47,NL,Nederland,"[Tooropstraat 47, Nijmegen, Nederland]",NaN,Nijmegen,Gelderland,NaN
3,4c4b2a5e5609c9b61175c390,Restaurant Indian Way,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1587886515,False,NaN,51.845811,5.871616,"[{'label': 'display', 'lat': 51.84581125268259...",1116,NL,Nederland,[Nederland],NaN,NaN,NaN,NaN
4,4b8f928df964a5207f5833e3,Restaurant Vesters,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",v-1587886515,False,Groesbeeksedwarsweg 307,51.835059,5.871045,"[{'label': 'display', 'lat': 51.83505901370137...",552,NL,Nederland,"[Groesbeeksedwarsweg 307 (Groesbeekseweg), 652...",6521DK,Nijmegen,Gelderland,Groesbeekseweg


In [28]:
dataframe.shape

(12, 17)

## Since I have exceeded the number of requests on the Foursquare API, I cannot check ratings.

# Location 2 has much more restaurant in its vicinity

## Conclusion: the restaurauteur could best open his new restaurant on location 1